
두번째 셀의 3번줄이 수집시간인데 720분으로 설정되어있어서 매일 아침11시에 실행시키시고, 오후 11시에 코드가 종료될겁니다

셀 실행전 매일 수정해주셔야 될것 하나:
두번째 셀에서 밑에서 두번째 줄에 저장파일명을 바꿔줘야 되는데 7000_요일.csv로 바꿔 주심 됩니다~

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import warnings
import time
import sklearn.preprocessing

pd.set_option("display.max_columns", None)
warnings.filterwarnings(action='ignore')

serviceKey='bwUUK6MEA6GnqDpo2ZS9ChN4VZ1nkUPXrryU80XxUcLqxB62pIGIPWXNdyzqgT20BHrw8zxWWk9rqiSQwTrbnQ=='

p7000 ={'serviceKey' :serviceKey,
             'routeId':'200000112','stationId':'228000703'}
class Bus():
    def __init__(self,params):
        self.params=params
        self.bus_time=[]
        
    def repet(self):
        url = 'http://apis.data.go.kr/6410000/busarrivalservice/getBusArrivalItem'
        response = requests.get(url, params=self.params)
        soup=BeautifulSoup(response.text,'xml')
        return soup

        
    def current_buses(self): 
        url = 'http://apis.data.go.kr/6410000/buslocationservice/getBusLocationList'
        mod_params ={'serviceKey' : self.params['serviceKey'],
                  'routeId':self.params['routeId']}

        response=requests.get(url,params=mod_params)
        soup=BeautifulSoup(response.text,'xml')
        locations=sorted([int(x.stationSeq.text) for x in soup.find_all('busLocationList')])
        return locations


def trim_data(df,FileName):
    df['시간'] = df['시간'].apply(pd.to_datetime)
    df['hour']=df['시간'].dt.hour
    df['minute']=df['시간'].dt.minute
    df['second']=df['시간'].dt.second
    df['day'] = df['시간'].dt.weekday 

    mb = sklearn.preprocessing.MultiLabelBinarizer()
    if isinstance(df['운행중인 다른 버스들의 위치'].iloc[0], str):
        df['운행중인 다른 버스들의 위치'] = df['운행중인 다른 버스들의 위치'].apply(eval)
    entity = mb.fit_transform(df['운행중인 다른 버스들의 위치']) 
    lsts=pd.DataFrame(entity,columns=mb.classes_)
    lsts.columns=[f'station{x}' for x in lsts.columns]
    df=pd.concat([df,lsts],axis=1,sort=False).reset_index(drop=True)


    df=df.drop(["운행중인 다른 버스들의 위치"],axis=1)
    
    batch=df.index[df.arrived==1]
    df=df.iloc[:batch[-1]+1,:]
    #7/12 -----------------------------------
    batch=df.index[df.arrived==1]
    temp=0
    for x in batch:
        for y in range(temp,x): 
            df['arrived'][y]=(df['시간'][x]-df['시간'][y]).seconds
            temp=x+1
    for e in batch:
        df['arrived'][e]=0
    # 7/12 -----------------------------------

    df = df.set_index('시간')
    df.to_csv(f'./{FileName}')        # 동일 폴더에 파일이 저장될 거임
    return df

In [2]:
past=4
pBus=Bus(p7000)
for x in range(720): # 시간 설정
    soup=pBus.repet()
    try:
        result=int(soup.resultCode.text)
        arrived=0
        if past!=result:
            if result==0:
                arrived=1
        pBus.bus_time.append([soup.queryTime.text,pBus.current_buses(),arrived])
        past=result
        print(x,"번째 시도: ",pBus.bus_time)
        time.sleep(60)
    except Exception as e:
        continue

df=pd.DataFrame(pBus.bus_time,columns=['시간','운행중인 다른 버스들의 위치','arrived'])

mod_df=trim_data(df,'7000_mon.csv') # 이름 설정
mod_df

0 번째 시도:  [['2022-08-02 09:43:23.853', [3, 14, 26, 32, 39, 50, 52, 56, 71], 0]]
1 번째 시도:  [['2022-08-02 09:43:23.853', [3, 14, 26, 32, 39, 50, 52, 56, 71], 0], ['2022-08-02 09:44:24.078', [5, 14, 27, 33, 40, 50, 52, 57, 71], 0]]
2 번째 시도:  [['2022-08-02 09:43:23.853', [3, 14, 26, 32, 39, 50, 52, 56, 71], 0], ['2022-08-02 09:44:24.078', [5, 14, 27, 33, 40, 50, 52, 57, 71], 0], ['2022-08-02 09:45:25.538', [5, 15, 27, 34, 40, 51, 52, 58, 73], 0]]
3 번째 시도:  [['2022-08-02 09:43:23.853', [3, 14, 26, 32, 39, 50, 52, 56, 71], 0], ['2022-08-02 09:44:24.078', [5, 14, 27, 33, 40, 50, 52, 57, 71], 0], ['2022-08-02 09:45:25.538', [5, 15, 27, 34, 40, 51, 52, 58, 73], 0], ['2022-08-02 09:46:31.481', [6, 16, 27, 34, 41, 51, 53, 60, 73], 0]]
4 번째 시도:  [['2022-08-02 09:43:23.853', [3, 14, 26, 32, 39, 50, 52, 56, 71], 0], ['2022-08-02 09:44:24.078', [5, 14, 27, 33, 40, 50, 52, 57, 71], 0], ['2022-08-02 09:45:25.538', [5, 15, 27, 34, 40, 51, 52, 58, 73], 0], ['2022-08-02 09:46:31.481', [6, 16, 27, 34, 41, 

15 번째 시도:  [['2022-08-02 09:43:23.853', [3, 14, 26, 32, 39, 50, 52, 56, 71], 0], ['2022-08-02 09:44:24.078', [5, 14, 27, 33, 40, 50, 52, 57, 71], 0], ['2022-08-02 09:45:25.538', [5, 15, 27, 34, 40, 51, 52, 58, 73], 0], ['2022-08-02 09:46:31.481', [6, 16, 27, 34, 41, 51, 53, 60, 73], 0], ['2022-08-02 09:47:32.643', [7, 16, 28, 36, 42, 51, 53, 60, 75], 0], ['2022-08-02 09:48:33.492', [8, 17, 28, 37, 43, 51, 53, 61, 76], 0], ['2022-08-02 09:49:35.232', [9, 18, 29, 37, 44, 52, 54, 62, 77], 0], ['2022-08-02 09:50:52.594', [9, 19, 29, 38, 44, 52, 55, 63, 79], 0], ['2022-08-02 09:51:52.985', [9, 19, 29, 38, 45, 52, 56, 63, 79], 0], ['2022-08-02 09:52:53.140', [10, 20, 30, 38, 45, 52, 56, 63], 0], ['2022-08-02 09:53:53.343', [10, 21, 31, 39, 46, 53, 57, 64], 0], ['2022-08-02 09:54:55.968', [10, 21, 31, 40, 46, 53, 59, 65], 0], ['2022-08-02 09:55:56.905', [11, 22, 32, 40, 47, 54, 60, 65], 0], ['2022-08-02 09:56:57.092', [11, 23, 33, 40, 48, 54, 61, 65], 0], ['2022-08-02 09:57:57.404', [12, 23, 

22 번째 시도:  [['2022-08-02 09:43:23.853', [3, 14, 26, 32, 39, 50, 52, 56, 71], 0], ['2022-08-02 09:44:24.078', [5, 14, 27, 33, 40, 50, 52, 57, 71], 0], ['2022-08-02 09:45:25.538', [5, 15, 27, 34, 40, 51, 52, 58, 73], 0], ['2022-08-02 09:46:31.481', [6, 16, 27, 34, 41, 51, 53, 60, 73], 0], ['2022-08-02 09:47:32.643', [7, 16, 28, 36, 42, 51, 53, 60, 75], 0], ['2022-08-02 09:48:33.492', [8, 17, 28, 37, 43, 51, 53, 61, 76], 0], ['2022-08-02 09:49:35.232', [9, 18, 29, 37, 44, 52, 54, 62, 77], 0], ['2022-08-02 09:50:52.594', [9, 19, 29, 38, 44, 52, 55, 63, 79], 0], ['2022-08-02 09:51:52.985', [9, 19, 29, 38, 45, 52, 56, 63, 79], 0], ['2022-08-02 09:52:53.140', [10, 20, 30, 38, 45, 52, 56, 63], 0], ['2022-08-02 09:53:53.343', [10, 21, 31, 39, 46, 53, 57, 64], 0], ['2022-08-02 09:54:55.968', [10, 21, 31, 40, 46, 53, 59, 65], 0], ['2022-08-02 09:55:56.905', [11, 22, 32, 40, 47, 54, 60, 65], 0], ['2022-08-02 09:56:57.092', [11, 23, 33, 40, 48, 54, 61, 65], 0], ['2022-08-02 09:57:57.404', [12, 23, 

28 번째 시도:  [['2022-08-02 09:43:23.853', [3, 14, 26, 32, 39, 50, 52, 56, 71], 0], ['2022-08-02 09:44:24.078', [5, 14, 27, 33, 40, 50, 52, 57, 71], 0], ['2022-08-02 09:45:25.538', [5, 15, 27, 34, 40, 51, 52, 58, 73], 0], ['2022-08-02 09:46:31.481', [6, 16, 27, 34, 41, 51, 53, 60, 73], 0], ['2022-08-02 09:47:32.643', [7, 16, 28, 36, 42, 51, 53, 60, 75], 0], ['2022-08-02 09:48:33.492', [8, 17, 28, 37, 43, 51, 53, 61, 76], 0], ['2022-08-02 09:49:35.232', [9, 18, 29, 37, 44, 52, 54, 62, 77], 0], ['2022-08-02 09:50:52.594', [9, 19, 29, 38, 44, 52, 55, 63, 79], 0], ['2022-08-02 09:51:52.985', [9, 19, 29, 38, 45, 52, 56, 63, 79], 0], ['2022-08-02 09:52:53.140', [10, 20, 30, 38, 45, 52, 56, 63], 0], ['2022-08-02 09:53:53.343', [10, 21, 31, 39, 46, 53, 57, 64], 0], ['2022-08-02 09:54:55.968', [10, 21, 31, 40, 46, 53, 59, 65], 0], ['2022-08-02 09:55:56.905', [11, 22, 32, 40, 47, 54, 60, 65], 0], ['2022-08-02 09:56:57.092', [11, 23, 33, 40, 48, 54, 61, 65], 0], ['2022-08-02 09:57:57.404', [12, 23, 

33 번째 시도:  [['2022-08-02 09:43:23.853', [3, 14, 26, 32, 39, 50, 52, 56, 71], 0], ['2022-08-02 09:44:24.078', [5, 14, 27, 33, 40, 50, 52, 57, 71], 0], ['2022-08-02 09:45:25.538', [5, 15, 27, 34, 40, 51, 52, 58, 73], 0], ['2022-08-02 09:46:31.481', [6, 16, 27, 34, 41, 51, 53, 60, 73], 0], ['2022-08-02 09:47:32.643', [7, 16, 28, 36, 42, 51, 53, 60, 75], 0], ['2022-08-02 09:48:33.492', [8, 17, 28, 37, 43, 51, 53, 61, 76], 0], ['2022-08-02 09:49:35.232', [9, 18, 29, 37, 44, 52, 54, 62, 77], 0], ['2022-08-02 09:50:52.594', [9, 19, 29, 38, 44, 52, 55, 63, 79], 0], ['2022-08-02 09:51:52.985', [9, 19, 29, 38, 45, 52, 56, 63, 79], 0], ['2022-08-02 09:52:53.140', [10, 20, 30, 38, 45, 52, 56, 63], 0], ['2022-08-02 09:53:53.343', [10, 21, 31, 39, 46, 53, 57, 64], 0], ['2022-08-02 09:54:55.968', [10, 21, 31, 40, 46, 53, 59, 65], 0], ['2022-08-02 09:55:56.905', [11, 22, 32, 40, 47, 54, 60, 65], 0], ['2022-08-02 09:56:57.092', [11, 23, 33, 40, 48, 54, 61, 65], 0], ['2022-08-02 09:57:57.404', [12, 23, 

ConnectionError: HTTPConnectionPool(host='apis.data.go.kr', port=80): Max retries exceeded with url: /6410000/busarrivalservice/getBusArrivalItem?serviceKey=bwUUK6MEA6GnqDpo2ZS9ChN4VZ1nkUPXrryU80XxUcLqxB62pIGIPWXNdyzqgT20BHrw8zxWWk9rqiSQwTrbnQ%3D%3D&routeId=200000112&stationId=228000703 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000025B3F06E7C0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))